In [ ]:
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU,Dense
from keras.layers import ZeroPadding2D,Flatten
from keras.layers import UpSampling2D,Lambda
from keras.layers import add, Concatenate,GlobalAveragePooling2D,Softmax
from keras.models import Model
import tensorflow as tf
from keras.applications import vgg19
import tensorflow_datasets as tfds
import os
import cv2 
from google.colab.patches import cv2_imshow
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip' --output 'DIV2K_train_LR_bicubic_X2.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/SRGAN/LRR")

In [ ]:
!unzip 'DIV2K_train_LR_bicubic_X2.zip' -d 'DATA'

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_unknown_X2.zip' --output 'DIV2K_train_LR_unknown_X2.zip'

In [ ]:
!unzip 'DIV2K_train_LR_unknown_X2.zip' -d 'DATA'

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip' --output 'DIV2K_train_HR.zip'

In [ ]:
!unzip 'DIV2K_train_HR.zip' -d 'train data'

In [ ]:
#Getting the path to all LR images having unknown downsampling
def getting_paths(dir_name,name):

# 
    Root=Path(dir_name)
    path= list(Root.glob('*.png'))

    values=[str(i+1) for i in range(len(path))]
    id=[]
    back_name='x2.png' if name=='LR' else '.png'

    for val in values:
       id.append((4-len(val))*'0'+val+back_name)
    path=[dir_name+i for i in id]
    return path



In [ ]:
LR_dir= 'DATA/DIV2K_train_LR_unknown/X2/'
HR_dir= 'train data/DIV2K_train_HR/'
path_to_LR= getting_paths(LR_dir,'LR')
print('The total LR image size',len(path_to_LR))
path_to_HR= getting_paths(HR_dir,'HR')
print('The toatl HR image size',len(path_to_HR))

In [ ]:
def augmentation(path_to_HR,HR_dir,path_to_LR,LR_dir):

    count=800 
    for i in range(800):

        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])

            imgflip_HR= cv2.flip(img_HR,0) #flipping the image from x-axis
            imgflip_LR= cv2.flip(img_LR,0)
            #Getting the location of the images
           
            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
    
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])
            
            #Rotating the image at an angle of 90
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_CLOCKWISE)
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_CLOCKWISE)
            #Getting the location of the images

            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
      
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])
            
            #Rotating the image at an angle of 270
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            
            #Getting the location of the images

            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
       
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)   




In [ ]:
augmentation(path_to_HR,HR_dir,path_to_LR,LR_dir)
path_to_LR= getting_paths(LR_dir,'LR')
print('The total LR image size after augmentation',len(path_to_LR))
path_to_HR= getting_paths(HR_dir,'HR')
print('The toatl HR image size after augmentation',len(path_to_HR))

In [ ]:
parent_dir = "/content/drive/MyDrive/SRGAN/5LR"
# directory=0
for a1 in range(35):
  # if a1%10 ==0:
    # directory +=1
  path = os.path.join(parent_dir, str(a1+1))
  os.mkdir(path)

In [ ]:
# @tf.function
def Total_patches(paths,patch_size,name):
    
    total_data=[]
    shapes_img=[]
    count=1
    for path in tqdm(paths):
         
        patch,sh,count=getting_patches(path,patch_size,name,count)
        # print(sh)
        total_data.extend(patch)
        shapes_img.extend(sh)
    loc='/content/drive/MyDrive/SRGAN/5LR'+'.pkl'   
    with open(loc,'wb') as f:
         pickle.dump(shapes_img,f)

#getting patches of the input images
def getting_patches(paths,patch_size,name,count):

    scale= 1 if name=='LR' else 2
    patch_size=patch_size*scale
    for i in range(1):
        patches=[]
        shapes=[]
        image= cv2.imread(paths, cv2.IMREAD_UNCHANGED)
        ny= image.shape[0]//patch_size
        nx= image.shape[1]//patch_size
        shapes.append([ny,nx])


        for i in range(ny):

          starty=i*patch_size

          for j in range(nx):

            start= j*patch_size
            end= patch_size + j*patch_size

            Lr_patch= image[starty:patch_size+starty,start:end]
            # Lr_patch= image[:patch_size,start:en
            # parent_dir="/content/drive/MyDrive/SRGAN/5LR/"
            # directory=1
            # if count%7500==0:
            #   directory +=1
            #   pathh = os.path.join(parent_dir, str(directory))
            #   os.mkdir(pathh)

            folder=1
            if count%7500 == 1:
              folder = + 1
            location='/content/drive/MyDrive/SRGAN/5LR/'+str(folder)+'/'+str(count)+'.png'
            # location= pathh +"/"+str(count)+'.png'

            count=count+1
            cv2.imwrite(location,Lr_patch)
            patches.append(Lr_patch)


    return patches,shapes,count


#combining the patches
def combine_patches(patches,shapes):
  
      h,w= shapes
      horizontal_image=[] 

      for i in range(h):
        image= np.hstack((patches[i*w],patches[i*w+1])) #combining first two patched images
        index= i*w+2 #As the first two patched imaged have been combined, now index selects third image

        for j in range(w-2): #-2 because the first the patches have been joined already
            image=np.hstack((image,patches[index+j]))
         
        horizontal_image.append(image) 
    
      actual_image=np.vstack((horizontal_image[0],horizontal_image[1]))
      for i in range(h-2):
        actual_image= np.vstack((actual_image,horizontal_image[2+i]))

      return actual_image    


def Total_combine_patches(patches,shapes,data_size,image_index=None):

  temporary= 0   
  total_actual_image=[]
  if image_index != None:
      image_index=image_index-1
      for i in range(image_index):
          hs= shapes[i][0]          
          ws= shapes[i][1]
          temporary= temporary+hs*ws
       
      h_,w_= shapes[image_index]
      return  combine_patches(patches[temporary:temporary+h_*w_],shapes[image_index])   


  for i in tqdm(range(data_size)):  
      h,w= shapes[i]
      image=combine_patches(patches[temporary:temporary+h*w],shapes[i])
      total_actual_image.append(image)
      temporary= h*w+temporary      

  return total_actual_image
    


In [ ]:
Total_patches(path_to_LR,48,'LR')

In [ ]:
a=os.listdir('/content/drive/My Drive/SRGAN/HR')
img= cv2.imread('/content/drive/My Drive/SRGAN/HR/5.png')
# cv2_imshow(img)
ar=np.empty(shape=(294,96,96,3))
for i in range(294):
    loc='drive/My Drive/SRGAN/HR' + a[i]
    print
    ar[i]= cv2.imread(loc)
